In [1]:
import numpy as np
from numpy import dot
from numpy.linalg import norm
from gensim.models.word2vec import Word2Vec
from konlpy.tag import Okt
import pandas as pd
import time

In [11]:
with open('C:\\Users\\ychn0\\OneDrive\\문서\\MyProgram\\Python\\ADCO\\datas\\review_data.txt', 'r', encoding='UTF-8') as file:
    content = file.read()
train_data = content.split('\n')[3:-1]
for ite in range(len(train_data)):
    train_data[ite] = train_data[ite].split('|')[1]
# 결손 데이터 제거
print(f"데이터 갯수 : {len(train_data)}")
temp = train_data.copy()
train_data.clear()
for ite in range(len(temp)):
    if (temp[ite] != ''):
        train_data.append(temp[ite])
print(f"결손 데이터 제거 후 데이터 갯수 : {len(train_data)}")
train_data = pd.Series(train_data)
print(train_data[:3])
train_data = train_data.str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","", regex=True)
print(train_data[:10])
# 불용어 정의
josa_file_name = "datas\\adco_data_josa.csv"
with open(josa_file_name, 'r', encoding = "UTF-8") as josa_file:
    stopwords = josa_file.read().split(',')

# 형태소 분석기 OKT를 사용한 토큰화 작업 (다소 시간 소요)
okt = Okt()
tokenized_data = []
for sentence in train_data:
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    tokenized_data.append(stopwords_removed_sentence)
with open("datas\\adco_data_words.csv",'r',encoding = "UTF-8") as file:
    guideline_words = file.read().split(',')
tokenized_data += guideline_words
print('리뷰의 최대 길이 :',max(len(review) for review in tokenized_data))
print('리뷰의 평균 길이 :',sum(map(len, tokenized_data))/len(tokenized_data))
print(tokenized_data[:4])

데이터 갯수 : 3052
결손 데이터 제거 후 데이터 갯수 : 2942
0    부모님 모시고 방문했어요 여의도파라곤이라 주차도 편했고 위치도 좋았습니다 룸형태라 ...
1    양 정말 푸짐하고 맛있었던 고방채 꼬막비빔밥 새우전 정식 먹었는데 2인분이 3인분같...
2    문어 숙회랑 냉수육이 맛있을수있는 음식인지 처음알았어요!! 기대도안했던 어묵탕......
dtype: object
0    부모님 모시고 방문했어요 여의도파라곤이라 주차도 편했고 위치도 좋았습니다 룸형태라 ...
1    양 정말 푸짐하고 맛있었던 고방채 꼬막비빔밥 새우전 정식 먹었는데 인분이 인분같이 ...
2    문어 숙회랑 냉수육이 맛있을수있는 음식인지 처음알았어요 기대도안했던 어묵탕맛있는어묵...
3    여기는 부모님 모시고 꼭 가보세요 프라이빗한 룸에서 친구들과 편히 수다떨면서 맛난 ...
4    아주아주 맛있게 잘 먹었습니다 직장 동료 들과 거하게 먹고 갑니다 역시 고방채 호점...
5    가족모임으로 갔는데 너무 좋았어요 룸도 적당히 넓고 깨끗하고 음식도 다 너무 맛있었...
6               너무 정갈하고 맛있었습니다  조만간 디너에 친구들과 또 방문하겠습니다
7    단독룸이라 회삭 비즈니시미팅 가족 모임하기 좋아요 가마솥 전복죽 먹었는데 전복죽도 ...
8             자주 오는 식당이어요 내일도 옵니다 맛있어요 솥밥의 밥이 촉촉하고 맛나요
9    여의도를 중심으로 모던한식을 추구하는 고방채  육회와 뭉티기 냉편육모두 보여지는 것...
dtype: object
리뷰의 최대 길이 : 121
리뷰의 평균 길이 : 14.405327342747112
[['부모님', '모시', '방문', '여의도', '파', '라곤', '이르다', '주차', '편하다', '위치', '좋다', '룸형태', '라', '프라이', '빗', '나오다', '음식', '비주', '얼', '퀄리티', '모두', '만족스럽다', '부모님', '께', '멋지다', '곳', '에서',

In [7]:
print("training start")
start = time.time()
model = Word2Vec(sentences = tokenized_data, vector_size = 100, window = 5, min_count = 5, workers = 4, sg = 0)
print("training finished :", time.time() - start)
print(model.wv.vectors.shape)

file_index = 0

model.save("datas\\w2v_model.bin")
print(f"model saved as datas\\w2v_model.bin")

training start
training finished : 0.31038403511047363
(1157, 100)
model saved as datas\w2v_model.bin


In [8]:
print(model.wv.most_similar("돈까스"))

[('좋아하다', 0.9996371269226074), ('많이', 0.9996271133422852), ('않다', 0.9996048808097839), ('시키다', 0.999603271484375), ('소스', 0.9995949864387512), ('제', 0.9995900988578796), ('구이', 0.9995846748352051), ('안', 0.9995818138122559), ('나다', 0.9995814561843872), ('나오다', 0.9995805025100708)]
